In [27]:
import cv2 as cv
import json
import os

In [28]:
FRAME_NUM = 100
FRAME_PATH = "Z:\CODE\YOLO-IMPLEMENT\IMAGES\TRAIN"
JSON_PATH = "Z:\CODE/adelaide-annotations.json"
OUT_PATH= "Z:\CODE\output"

f = open(JSON_PATH)
data = json.load(f)
frames = sorted(os.listdir(FRAME_PATH))
FRAME_DIM = (data['item']['slots'][0]['width'], data['item']['slots'][0]['height'])

print(f'frame dimensions: {FRAME_DIM}')

all_files = os.listdir(FRAME_PATH)
files = [f for f in all_files if os.path.isfile(os.path.join(FRAME_PATH, f))]
    


frame dimensions: (1920, 1080)


In [29]:
classes = [
    'outer-goal-square-bottom-left', 'outer-goal-square-top-left',
    'inner-goal-square-bottom-left', 'inner-goal-square-top-left',
    '50-line-bottom-left', '50-line-top-left',
    'center-square-bottom-left', 'center-square-top-left',
    'outer-circle-bottom', 'inner-circle-bottom',
    'inner-circle-top', 'outer-circle-top',
    'center-square-bottom-right', 'center-square-top-right',
    '50-line-bottom-right', '50-line-top-right',
    'inner-goal-square-bottom-right', 'inner-goal-square-top-right',
    'outer-goal-square-bottom-right', 'outer-goal-square-top-right'
]

In [30]:
def get_keypoints_for_frame(data, frame_num,txt_file_name,OUT_PATH):

    file_path = os.path.join(OUT_PATH, txt_file_name)

    keypoints = []
    for annotation in data:
        frames = annotation.get('frames', {})
        frame = frames.get(str(frame_num), {})
        keypoint = frame.get('keypoint', None)

        if keypoint:
            keypoints.append({
                'name': annotation.get('name'),
                'keypoint': keypoint
            })

    keypoints_dict = {kp['name']: {**kp['keypoint']} for kp in keypoints}
    final_keypoints = []
    for name in classes:
        if name in keypoints_dict:
            x = keypoints_dict[name]['x']
            y = keypoints_dict[name]['y']
            final_keypoints.append({'name': name, 'keypoint': (x, y, 2)})
        else:
            final_keypoints.append({'name': name, 'keypoint': (-1, -1, 0)})

    bounding_boxes = []
    for annotation in data:
        frames = annotation.get('frames', {})
        frame = frames.get(str(frame_num), {})
        bounding_box = frame.get('bounding_box', None)

        if bounding_box:
            bounding_boxes.append({
                'name': annotation.get('name'),
                'bounding_box': bounding_box
            })

    final_bounding_boxes = []
    with open(file_path, 'w') as file:
        for bb in bounding_boxes:
            name = bb['name']
            x = round(bb['bounding_box']['x'])
            y = round(bb['bounding_box']['y'])
            w = round(bb['bounding_box']['w'])
            h = round(bb['bounding_box']['h'])
            final_bounding_boxes.append({'name': name, 'bounding_box': (x, y, w, h)})

            # Calculate xtl, ytl, xbr, ybr
            xtl = x
            ytl = y
            xbr = x + w
            ybr = y + h

            # Normalize coordinates
            image_width = 1920
            image_height = 1080
            normalized_xtl = xtl / image_width
            normalized_ytl = ytl / image_height
            normalized_xbr = xbr / image_width
            normalized_ybr = ybr / image_height

            normalized_w = (xbr - xtl) / image_width
            normalized_h = (ybr - ytl) / image_height
            normalized_center_x = (normalized_xtl + normalized_w / 2)
            normalized_center_y = (normalized_ytl + normalized_h / 2)

            # Write bounding box data to file
            file.write(f'0 {normalized_center_x} {normalized_center_y} {normalized_w} {normalized_h}\n')

        # Process keypoints
        for kp in final_keypoints:
            name = kp['name']
            x, y, visibility = kp['keypoint']

            # Normalize keypoint coordinates
            normalized_x = x / image_width
            normalized_y = y / image_height

            # Write keypoint data to file
            file.write(f' {normalized_x} {normalized_y} {visibility}\n')



    return final_keypoints, final_bounding_boxes


In [31]:
# img = cv.imread(os.path.join(FRAME_PATH, frames[FRAME_NUM]))


for FRAME_NUM in range(len(files)):
    imgname = frames[FRAME_NUM]
    txt_file_name = os.path.splitext(imgname)[0] + '.txt'
    keypoints_data, bounding_boxes_data = get_keypoints_for_frame(data['annotations'], FRAME_NUM, txt_file_name, OUT_PATH)


# print(f'number of keypoints: {len(keypoints_data)}')
# print(f'image shape: {img.shape}')

# print('\nkeypoints')
# for i in range(len(keypoints_data)):
#     print(f'{i}: {keypoints_data[i]}')



# print('\nbounding boxes')
# for i in range(len(bounding_boxes_data)):
#     print(f'{i}: {bounding_boxes_data[i]}')
